<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

In [2]:
# Start session
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

In [4]:
spark = SparkSession.builder.appName("Capstone").getOrCreate()

25/05/02 16:14:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2025-05-02 16:14:22--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.1’

searchterms.csv.1   100%[===================>] 227.99K  --.-KB/s    in 0.004s  

2025-05-02 16:14:22 (55.6 MB/s) - ‘searchterms.csv.1’ saved [233457/233457]



In [8]:
df = spark.read.csv("searchterms.csv",header=True,inferSchema=True)

In [13]:
rows = df.count()
columns = len(df.columns)
print("rows: " + str(rows) + " columns: " + str(columns))

rows: 10000 columns: 4


In [14]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm:

In [25]:
df.dtypes[-1]

('searchterm', 'string')

In [ ]:
# How many times was the term `gaming laptop` searched:

In [40]:
df.filter("searchterm == 'gaming laptop'").count()

499

In [50]:
from pyspark.sql.functions import count
df.groupBy("searchterm").agg(count("searchterm").alias("count")).sort("count", ascending = False).show(5)

[Stage 51:=================================>                    (124 + 9) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [53]:
!wget  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xf model.tar.gz

In [54]:
!ls

Spark_MLOps.ipynb  sales_prediction.model  searchterms.csv.1
model.tar.gz	   searchterms.csv


In [70]:
# Using the sales forecast model, predict the sales for the year of 2023:
from pyspark.ml.regression import LinearRegressionModel

In [71]:
loadedModel = LinearRegressionModel.load("sales_prediction.model")

In [105]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    year_df =spark.createDataFrame([[year]],["year"])
    assembled_df = assembler.transform(year_df).select("features")
    assembled_df.show()
    predictions = loadedModel.transform(assembled_df)
    predictions.select('prediction').show()


In [106]:
predict(2023)

+--------+
|features|
+--------+
|[2023.0]|
+--------+

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



In [82]:
indexer = StringIndexer(inputCol = "searchterm", outputCol = "label")

In [98]:
.show()

+----+
|year|
+----+
|2023|
+----+

